In [227]:
!pip install feature_engine
import pandas as pd
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from feature_engine.encoding import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from feature_engine import imputation
from sklearn.experimental import enable_iterative_imputer
from xgboost import XGBRegressor
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [228]:
# To Import Data from google drive (authentication needed) 

# Importing train.csv data
from google.colab import drive 
drive.mount('/content/gdrive')

Data = pd.read_csv('gdrive/MyDrive/Personal Data Science Projects/Big Mart Sales Predictions Analytics Vidhya/train.csv')
Data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [229]:
# Importing test.csv data

Test_Data = pd.read_csv('gdrive/MyDrive/Personal Data Science Projects/Big Mart Sales Predictions Analytics Vidhya/test.csv')
Test_Data_identifier = Test_Data[['Item_Identifier', 'Outlet_Identifier']]
Test_Data.drop(columns=['Item_Identifier', 'Outlet_Identifier'], inplace=True)
Test_Data

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.750,Low Fat,0.007565,Snack Foods,107.8622,1999,Medium,Tier 1,Supermarket Type1
1,8.300,reg,0.038428,Dairy,87.3198,2007,NaN,Tier 2,Supermarket Type1
2,14.600,Low Fat,0.099575,Others,241.7538,1998,NaN,Tier 3,Grocery Store
3,7.315,Low Fat,0.015388,Snack Foods,155.0340,2007,NaN,Tier 2,Supermarket Type1
4,NaN,Regular,0.118599,Dairy,234.2300,1985,Medium,Tier 3,Supermarket Type3
...,...,...,...,...,...,...,...,...,...
5676,10.500,Regular,0.013496,Snack Foods,141.3154,1997,Small,Tier 1,Supermarket Type1
5677,7.600,Regular,0.142991,Starchy Foods,169.1448,2009,Medium,Tier 3,Supermarket Type2
5678,10.000,Low Fat,0.073529,Health and Hygiene,118.7440,2002,NaN,Tier 2,Supermarket Type1
5679,15.300,Regular,0.000000,Canned,214.6218,2007,NaN,Tier 2,Supermarket Type1


In [230]:
df = Data.copy()
cat_features = ['Item_Fat_Content', 'Item_Type', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
num_features = list( set( Data.columns ) - set( cat_features ) )
num_features = ['Item_Weight','Item_Visibility','Item_MRP']
df_identifier = Data[['Item_Identifier', 'Outlet_Identifier']]
df.drop( columns = ['Item_Identifier', 'Outlet_Identifier'], inplace=True )
df_target_feature = df[['Item_Outlet_Sales']]

In [231]:
# Feature--> Item_Fat_Content | Catagorical | Ranked | treating it as not-ranked
df['Item_Fat_Content'].replace(to_replace=['low fat','LF'],value='Low Fat', inplace=True )
df['Item_Fat_Content'].replace(to_replace=['reg'],value= 'Regular', inplace=True )

# df['Item_Fat_Content'].unique() ----> # array(['Low Fat', 'Regular'], dtype=object)

one_hot = OneHotEncoder(drop_last=True)
df['Item_Fat_Content'] = one_hot.fit_transform(df[['Item_Fat_Content']])
Test_Data['Item_Fat_Content'] = one_hot.transform( Test_Data[['Item_Fat_Content']] )

In [232]:
# Feature--> Item_Type | Catagorical | Not-Ranked
df['Item_Type'].unique()
one_hot = OneHotEncoder(drop_last=False)
encoded_df = one_hot.fit_transform( df[['Item_Type']] )
encoded_df.drop(columns=['Item_Type_Others'],inplace=True)
df = df.join(encoded_df)
df.drop(columns='Item_Type',inplace=True)

encoded_df = one_hot.transform( Test_Data[['Item_Type']] )
encoded_df.drop(columns=['Item_Type_Others'],inplace=True)
Test_Data = Test_Data.join(encoded_df)
Test_Data.drop(columns='Item_Type',inplace=True)

In [233]:
# Feature--> Outlet_Location_Type | Catagorical | Ranked
# df['Outlet_Location_Type'].unique() ---> ['Tier 1', 'Tier 3', 'Tier 2']

ord_end = OrdinalEncoder(categories=[['Tier 3', 'Tier 2', 'Tier 1']])
df['Outlet_Location_Type'] = pd.DataFrame(ord_end.fit_transform(df[['Outlet_Location_Type']]),columns=['Outlet_Location_Type'])
Test_Data['Outlet_Location_Type'] = pd.DataFrame(ord_end.fit_transform(Test_Data[['Outlet_Location_Type']]),columns=['Outlet_Location_Type'])

In [234]:
# Feature--> Outlet_Type | Catagorical | Not-Ranked

# df['Outlet_Type'].unique() ---> ['Supermarket Type1', 'Supermarket Type2', 'Grocery Store', 'Supermarket Type3']

one_hot = OneHotEncoder(drop_last=False)
encoded_df = one_hot.fit_transform( df[['Outlet_Type']] )
df = df.join(encoded_df)
df.drop(columns='Outlet_Type',inplace=True)

encoded_df = one_hot.transform( Test_Data[['Outlet_Type']] )
Test_Data = Test_Data.join(encoded_df)
Test_Data.drop(columns='Outlet_Type',inplace=True)

In [235]:
 df.isnull().sum()

Item_Weight                        1463
Item_Fat_Content                      0
Item_Visibility                       0
Item_MRP                              0
Outlet_Establishment_Year             0
Outlet_Size                        2410
Outlet_Location_Type                  0
Item_Outlet_Sales                     0
Item_Type_Dairy                       0
Item_Type_Soft Drinks                 0
Item_Type_Meat                        0
Item_Type_Fruits and Vegetables       0
Item_Type_Household                   0
Item_Type_Baking Goods                0
Item_Type_Snack Foods                 0
Item_Type_Frozen Foods                0
Item_Type_Breakfast                   0
Item_Type_Health and Hygiene          0
Item_Type_Hard Drinks                 0
Item_Type_Canned                      0
Item_Type_Breads                      0
Item_Type_Starchy Foods               0
Item_Type_Seafood                     0
Outlet_Type_Supermarket Type1         0
Outlet_Type_Supermarket Type2         0


In [236]:
df.columns.values

array(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Item_Outlet_Sales', 'Item_Type_Dairy', 'Item_Type_Soft Drinks',
       'Item_Type_Meat', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Household', 'Item_Type_Baking Goods',
       'Item_Type_Snack Foods', 'Item_Type_Frozen Foods',
       'Item_Type_Breakfast', 'Item_Type_Health and Hygiene',
       'Item_Type_Hard Drinks', 'Item_Type_Canned', 'Item_Type_Breads',
       'Item_Type_Starchy Foods', 'Item_Type_Seafood',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3'],
      dtype=object)

In [237]:
Test_Data.isna().sum()

Item_Weight                         976
Item_Fat_Content                      0
Item_Visibility                       0
Item_MRP                              0
Outlet_Establishment_Year             0
Outlet_Size                        1606
Outlet_Location_Type                  0
Item_Type_Dairy                       0
Item_Type_Soft Drinks                 0
Item_Type_Meat                        0
Item_Type_Fruits and Vegetables       0
Item_Type_Household                   0
Item_Type_Baking Goods                0
Item_Type_Snack Foods                 0
Item_Type_Frozen Foods                0
Item_Type_Breakfast                   0
Item_Type_Health and Hygiene          0
Item_Type_Hard Drinks                 0
Item_Type_Canned                      0
Item_Type_Breads                      0
Item_Type_Starchy Foods               0
Item_Type_Seafood                     0
Outlet_Type_Supermarket Type1         0
Outlet_Type_Supermarket Type2         0
Outlet_Type_Grocery Store             0


### Missing Value Imputation using Random Forest | Numerical Data  --------> | It is not improving models performance |

In [238]:
# item_features = ['Item_Fat_Content','Item_Visibility', 'Item_MRP', 
#                 'Item_Type_Dairy', 'Item_Type_Soft Drinks',
#                 'Item_Type_Meat', 'Item_Type_Fruits and Vegetables',
#                 'Item_Type_Household', 'Item_Type_Baking Goods',
#                 'Item_Type_Snack Foods', 'Item_Type_Frozen Foods',
#                 'Item_Type_Breakfast', 'Item_Type_Health and Hygiene',
#                 'Item_Type_Hard Drinks', 'Item_Type_Canned', 'Item_Type_Breads',
#                 'Item_Type_Starchy Foods', 'Item_Type_Seafood',
#                 'Item_Weight']

# df_item = df[ item_features ]
              
# RFR = RandomForestRegressor(bootstrap= False, max_depth= None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2 , n_estimators=400 )

# imputer_RFR = IterativeImputer( estimator=RFR, max_iter=3, random_state=0 ) 

# imputer_RFR.fit(df_item)
# imputed_df_item = imputer_RFR.transform(df_item)

# imputed_df_item = pd.DataFrame(imputed_df_item, columns=df_item.columns)

# Test_Data_item = Test_Data[ item_features ]
# imputed_Test_Data_item = imputer_RFR.transform(Test_Data_item)
# imputed_Test_Data_item = pd.DataFrame(imputed_Test_Data_item, columns=Test_Data_item.columns)
# imputed_Test_Data_item

# Missing Value Imputation using Random Forest | Catagorical Data  --------> | It is not improving models performance |

In [239]:
# df_outlet = df[ [  'Outlet_Establishment_Year', 'Outlet_Location_Type',
#                     'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
#                     'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3','Outlet_Size'] ]

# df_outlet['Outlet_Size'].replace( to_replace='Small', value=0, inplace=True )
# df_outlet['Outlet_Size'].replace( to_replace='Medium', value=1, inplace=True )
# df_outlet['Outlet_Size'].replace( to_replace='High', value=2, inplace=True )

# rf_base = RandomForestClassifier( bootstrap= True,
#  criterion= 'entropy',
#  max_depth= 1750,
#  min_samples_leaf= 1,
#  min_samples_split= 5,
#  n_estimators= 2157 )

# imputer_RFC = IterativeImputer( estimator=rf_base, max_iter=3, random_state=0 ) 

# imputer_RFC.fit(df_outlet)
# imputed_df_outlet = imputer_RFC.fit_transform(df_outlet)

# imputed_df_outlet = pd.DataFrame(imputed_df_outlet, columns=df_outlet.columns)
# imputed_df_outlet

# Test_Data_outlet = df[ [  'Outlet_Establishment_Year', 'Outlet_Location_Type',
#                     'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
#                     'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3','Outlet_Size'] ]

# Test_Data_outlet['Outlet_Size'].replace( to_replace='Small', value=0, inplace=True )
# Test_Data_outlet['Outlet_Size'].replace( to_replace='Medium', value=1, inplace=True )
# Test_Data_outlet['Outlet_Size'].replace( to_replace='High', value=2, inplace=True )

# imputed_Test_Data_outlet = imputer_RFC.fit_transform(Test_Data_outlet)
# imputed_Test_Data_outlet = pd.DataFrame(imputed_Test_Data_outlet, columns=Test_Data_outlet.columns)
# imputed_Test_Data_outlet

##### Merge the two imputed datasets


In [240]:
# df = imputed_df_item.join(imputed_df_outlet)
# df = df.join(df_target_feature)

# Test_Data = imputed_Test_Data_item.join(imputed_Test_Data_outlet)
# Test_Data = Test_Data.join(Test_Data_target_feature)

***

# Imputation with Mean and Mode

In [241]:
X = df.drop(columns=['Item_Outlet_Sales'])
y = df[['Item_Outlet_Sales']]
X_train, X_test, y_train, y_test = train_test_split( X , y, test_size=0.1, random_state=42)

In [242]:
from sklearn.impute import SimpleImputer

df_outlet = X_train[ [  'Outlet_Establishment_Year', 'Outlet_Location_Type',
                    'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
                    'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3','Outlet_Size'] ]

df_outlet['Outlet_Size'].replace( to_replace='Small', value=0, inplace=True )
df_outlet['Outlet_Size'].replace( to_replace='Medium', value=1, inplace=True )
df_outlet['Outlet_Size'].replace( to_replace='High', value=2, inplace=True )

imputer_cat = SimpleImputer(strategy = 'most_frequent')

imputer_cat.fit(df_outlet)
imputed_df_outlet = imputer_cat.fit_transform(df_outlet)
imputed_df_outlet = pd.DataFrame(imputed_df_outlet, columns=df_outlet.columns)

df_outlet_test = X_test[ [  'Outlet_Establishment_Year', 'Outlet_Location_Type',
                    'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
                    'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3','Outlet_Size'] ]

df_outlet_test['Outlet_Size'].replace( to_replace='Small', value=0, inplace=True )
df_outlet_test['Outlet_Size'].replace( to_replace='Medium', value=1, inplace=True )
df_outlet_test['Outlet_Size'].replace( to_replace='High', value=2, inplace=True )

imputed_df_outlet_test = imputer_cat.transform(df_outlet_test)
imputed_df_outlet_test = pd.DataFrame(imputed_df_outlet_test, columns=df_outlet_test.columns)

Test_Data_outlet = Test_Data[ [  'Outlet_Establishment_Year', 'Outlet_Location_Type',
                    'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
                    'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type3','Outlet_Size' ]]

Test_Data_outlet['Outlet_Size'].replace( to_replace='Small', value=0, inplace=True )
Test_Data_outlet['Outlet_Size'].replace( to_replace='Medium', value=1, inplace=True )
Test_Data_outlet['Outlet_Size'].replace( to_replace='High', value=2, inplace=True )

imputed_Test_Data_outlet = imputer_cat.transform(Test_Data_outlet)
imputed_Test_Data_outlet = pd.DataFrame(imputed_Test_Data_outlet, columns=Test_Data_outlet.columns)

In [243]:
item_features = ['Item_Fat_Content','Item_Visibility', 'Item_MRP', 
                'Item_Type_Dairy', 'Item_Type_Soft Drinks',
                'Item_Type_Meat', 'Item_Type_Fruits and Vegetables',
                'Item_Type_Household', 'Item_Type_Baking Goods',
                'Item_Type_Snack Foods', 'Item_Type_Frozen Foods',
                'Item_Type_Breakfast', 'Item_Type_Health and Hygiene',
                'Item_Type_Hard Drinks', 'Item_Type_Canned', 'Item_Type_Breads',
                'Item_Type_Starchy Foods', 'Item_Type_Seafood',
                'Item_Weight']

df_item = X_train[ item_features ]

df_item['Item_Visibility'].replace(to_replace=0.0, value=np.nan,inplace=True) # Since any Item's Visibility cannot be zero having finite sales

imputer_num = SimpleImputer(strategy = 'mean')

imputer_num.fit(df_item)
imputed_df_item = imputer_num.fit_transform(df_item)
imputed_df_item = pd.DataFrame(imputed_df_item, columns=df_item.columns)

test_Data_item = X_test[ item_features ]
test_Data_item['Item_Visibility'].replace(to_replace=0.0, value=np.nan,inplace=True) #  Since any Item's Visibility cannot be zero having finite sales
imputed_test_Data_item = imputer_num.transform(test_Data_item)
imputed_test_Data_item = pd.DataFrame(imputed_test_Data_item, columns=test_Data_item.columns)

Test_Data_item = Test_Data[ item_features ]
Test_Data_item['Item_Visibility'].replace(to_replace=0.0, value=np.nan,inplace=True) #  Since any Item's Visibility cannot be zero having finite sales
imputed_Test_Data_item = imputer_num.transform(Test_Data_item)
imputed_Test_Data_item = pd.DataFrame(imputed_Test_Data_item, columns=Test_Data_item.columns)

In [244]:
X_train = imputed_df_item.join(imputed_df_outlet)

X_test = pd.concat( [imputed_test_Data_item , imputed_df_outlet_test], axis='columns' )

Test_Data = pd.concat( [imputed_Test_Data_item , imputed_Test_Data_outlet], axis='columns' )

In [245]:
X_train

,Item_Fat_Content,Item_Visibility,Item_MRP,Item_Type_Dairy,Item_Type_Soft Drinks,Item_Type_Meat,Item_Type_Fruits and Vegetables,Item_Type_Household,Item_Type_Baking Goods,Item_Type_Snack Foods,...,Item_Type_Starchy Foods,Item_Type_Seafood,Item_Weight,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type3,Outlet_Size
0,0.0,0.075142,145.8444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.881584,1985.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1.0,0.095696,159.7604,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,15.100000,2007.0,1.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.073667,66.2142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,11.150000,2009.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.025051,97.1410,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,16.100000,2009.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.050657,249.9092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.000000,1998.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665,0.0,0.286345,139.1838,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,9.395000,1998.0,0.0,0.0,0.0,1.0,0.0,1.0
7666,1.0,0.117575,75.6670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.600000,2007.0,1.0,1.0,0.0,0.0,0.0,1.0
7667,1.0,0.018944,237.3590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,17.600000,2002.0,1.0,1.0,0.0,0.0,0.0,1.0
7668,1.0,0.054363,117.9466,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,20.350000,2007.0,1.0,1.0,0.0,0.0,0.0,1.0


In [246]:
X_test

,Item_Fat_Content,Item_Visibility,Item_MRP,Item_Type_Dairy,Item_Type_Soft Drinks,Item_Type_Meat,Item_Type_Fruits and Vegetables,Item_Type_Household,Item_Type_Baking Goods,Item_Type_Snack Foods,...,Item_Type_Starchy Foods,Item_Type_Seafood,Item_Weight,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type3,Outlet_Size
0,1.0,0.026300,79.4302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,14.300000,1987.0,0.0,1.0,0.0,0.0,0.0,2.0
1,1.0,0.071136,42.7086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.930000,1997.0,2.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.041313,42.0454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,14.500000,1999.0,2.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.044767,173.7054,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.881584,1985.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.012456,197.5110,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.195000,2004.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,1.0,0.018814,237.9880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.270000,1997.0,2.0,1.0,0.0,0.0,0.0,0.0
849,1.0,0.073307,155.0972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.000000,1999.0,2.0,1.0,0.0,0.0,0.0,1.0
850,1.0,0.055944,150.6024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,15.700000,1987.0,0.0,1.0,0.0,0.0,0.0,2.0
851,1.0,0.053380,56.7904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.695000,1999.0,2.0,1.0,0.0,0.0,0.0,1.0


### Discrtization using KMeans --------> | It is not improving models performance |

In [247]:
# from sklearn.preprocessing import KBinsDiscretizer

# disc = KBinsDiscretizer(n_bins=25, encode='ordinal', strategy='kmeans')
# disc.fit(X_train[['Item_MRP']])
# print( disc.bin_edges_ )

# X_train['Item_MRP'] = disc.transform( X_train[['Item_MRP']] )
# X_test['Item_MRP'] = disc.transform(X_test[['Item_MRP']] )

# import plotly.figure_factory as ff

# hist_data = [X_train['Item_MRP']]

# group_labels = ['Item MRP'] # name of the dataset
# fig = ff.create_distplot(hist_data, group_labels)
# fig.show()

***

# Applying Models

In [248]:
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV

# params = { 'max_depth': [3,6,9],
#            'learning_rate': [0.01, 0.05, 0.1],
#            'n_estimators': [100,500,750,1000],
#            'colsample_bytree': [0.3, 0.7]}

# xgbr = xgb.XGBRegressor()
# clf = GridSearchCV(estimator=xgbr, 
#                    param_grid=params,
#                    scoring='neg_mean_squared_error', 
#                    verbose=1)
# clf.fit(X_train, y_train)
# print("Best parameters:", clf.best_params_)
# print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

In [249]:
# Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
# Lowest RMSE:  1092.9243292400545

# ---> 'Best Parameters" calculated by GridSearchCV is given negative values of target variable which is not practically possible --> Therefore, we reduce the n_estimator (to 50) and remove rest of the parameters

In [250]:
xgb_reg = XGBRegressor(n_estimators=50)
xgb_reg.fit(X_train, y_train)

y_predictions = xgb_reg.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
score = xgb_reg.score(X_test,y_test)
print('Root Mean Square Error : ', rmse )
print('Accuracy Score : ', score )

[13:28:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Square Error :  1028.5950119114627
Accuracy Score :  0.6215798004563415


## Feature Importance

In [251]:
df_FI = pd.DataFrame()
df_FI['column_names'] = X_train.columns
df_FI['FI_value'] = xgb_reg.feature_importances_
df_FI.sort_values(by='FI_value',inplace=True)
import plotly.express as px
fig = px.bar( x = df_FI['FI_value'], y=df_FI['column_names'] )
fig.update_layout( title='Feature Importance' )
fig.show()

In [252]:
fig = px.box( y = df['Outlet_Establishment_Year'] )
fig.show()

# Outlier are not present in this features

In [253]:
fig = px.box( y = Data['Item_MRP'] )
fig.show()

# Outlier are not present in this features

In [254]:
# Distplot of Item_MRP feature | Discretization is not imporving models perfomance

import plotly.figure_factory as ff

hist_data = [X_train['Item_MRP']]

group_labels = ['Item MRP'] # name of the dataset
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

***
#Final Predictions and Submission File Generation

In [255]:
X_train_final = pd.concat( [X_train, X_test], axis = 'rows' )
y_train_final = pd.concat( [y_train, y_test], axis = 'rows' )

In [256]:
xgb_reg = XGBRegressor(n_estimators= 50)
xgb_reg.fit(X_train_final, y_train_final)

y_pred_XGBReg = xgb_reg.predict(Test_Data)

[13:28:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [257]:
final_predictions_df = pd.DataFrame()
final_predictions_df['Item_Outlet_Sales'] = y_pred_XGBReg

In [258]:
Submission_Data = pd.concat([Test_Data_identifier,final_predictions_df], axis='columns')
Submission_Data

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1654.209106
1,FDW14,OUT017,1386.335449
2,NCN55,OUT010,603.356201
3,FDQ58,OUT017,2499.434814
4,FDY38,OUT027,6057.424316
...,...,...,...
5676,FDB58,OUT046,2129.620605
5677,FDD47,OUT018,2376.695312
5678,NCO17,OUT045,1882.120972
5679,FDJ26,OUT017,3556.062988


In [259]:
from google.colab import files

Submission_Data.to_csv('BigMartSalesPredictions_Shirsh_Submission_File.csv', encoding = 'utf-8-sig', index=False) 
files.download('BigMartSalesPredictions_Shirsh_Submission_File.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>